In [1]:
import pandas as pd
import numpy as np

## 7 数据清洗和准备
### 7.1 处理缺失数据

In [3]:
string_data = pd.Series(['aardvark','artichoke',np.nan,'avocado'])
string_data
#NaN相当于哨兵值

0     aardvark
1    artichoke
2          NaN
3      avocado
dtype: object

In [4]:
string_data.isnull()

0    False
1    False
2     True
3    False
dtype: bool

In [5]:
#python内置的None值在对象数组中也可以作为NA
string_data[0] = None
string_data.isnull()

0     True
1    False
2     True
3    False
dtype: bool

#### 滤除缺失数据

In [6]:
from numpy import nan as NA
data = pd.Series([1, NA, 3.5, NA, 7])
data.dropna()

0    1.0
2    3.5
4    7.0
dtype: float64

In [7]:
# above equal to this
data[data.notnull()]

0    1.0
2    3.5
4    7.0
dtype: float64

In [20]:
data = pd.DataFrame([[1., 6.5, 3.],[1., NA, NA],
                    [NA, NA, NA],[NA, 6.5, 3.]])
cleaned = data.dropna()
print(data)
print(cleaned)

     0    1    2
0  1.0  6.5  3.0
1  1.0  NaN  NaN
2  NaN  NaN  NaN
3  NaN  6.5  3.0
     0    1    2
0  1.0  6.5  3.0


In [21]:
# 传入how=’all‘将只丢弃全为NA的那些行
data.dropna(how='all')

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
3,NaN,6.5,3.0


In [22]:
#想丢弃列，则只需传入axis=1即可
data[4] = NA
print(data)
print(data.dropna(axis=1, how='all'))

     0    1    2   4
0  1.0  6.5  3.0 NaN
1  1.0  NaN  NaN NaN
2  NaN  NaN  NaN NaN
3  NaN  6.5  3.0 NaN
     0    1    2
0  1.0  6.5  3.0
1  1.0  NaN  NaN
2  NaN  NaN  NaN
3  NaN  6.5  3.0


In [30]:
#另一个滤除DataFrame行的问题涉及时间序列数据
#假设你只想留下一部分观测数据，可以用thresh参数实现此目的
df = pd.DataFrame(np.random.randn(7,3))
df.iloc[:4, 1] = NA
df.iloc[:2, 2] = NA
print(df)
print(df.dropna())
print(df.dropna(thresh=2))#thresh=2的意思是指至少有2个非NAN才能存活

          0         1         2
0  0.227816       NaN       NaN
1 -0.107160       NaN       NaN
2 -1.802504       NaN -0.339334
3 -1.339007       NaN -1.856904
4  0.708036 -0.333315  0.768803
5  0.296529  0.287088 -0.276000
6 -0.084794 -1.016629  0.205400
          0         1         2
4  0.708036 -0.333315  0.768803
5  0.296529  0.287088 -0.276000
6 -0.084794 -1.016629  0.205400
          0         1         2
2 -1.802504       NaN -0.339334
3 -1.339007       NaN -1.856904
4  0.708036 -0.333315  0.768803
5  0.296529  0.287088 -0.276000
6 -0.084794 -1.016629  0.205400


#### 填充缺失数据

In [31]:
#不滤除缺失数据，填补“空洞”，用fillna方法
print(df.fillna(0))
print(df.fillna({1: 0.5, 2: 0}))#第一列补0.5，第二列补0

          0         1         2
0  0.227816  0.000000  0.000000
1 -0.107160  0.000000  0.000000
2 -1.802504  0.000000 -0.339334
3 -1.339007  0.000000 -1.856904
4  0.708036 -0.333315  0.768803
5  0.296529  0.287088 -0.276000
6 -0.084794 -1.016629  0.205400
          0         1         2
0  0.227816  0.500000  0.000000
1 -0.107160  0.500000  0.000000
2 -1.802504  0.500000 -0.339334
3 -1.339007  0.500000 -1.856904
4  0.708036 -0.333315  0.768803
5  0.296529  0.287088 -0.276000
6 -0.084794 -1.016629  0.205400


In [32]:
#fillna默认会返回新对象，但也可以对现有对象进行就地修改
print(df)
_ = df.fillna(0, inplace=True)
df

          0         1         2
0  0.227816       NaN       NaN
1 -0.107160       NaN       NaN
2 -1.802504       NaN -0.339334
3 -1.339007       NaN -1.856904
4  0.708036 -0.333315  0.768803
5  0.296529  0.287088 -0.276000
6 -0.084794 -1.016629  0.205400


,0,1,2
0,0.227816,0.000000,0.000000
1,-0.107160,0.000000,0.000000
2,-1.802504,0.000000,-0.339334
3,-1.339007,0.000000,-1.856904
4,0.708036,-0.333315,0.768803
5,0.296529,0.287088,-0.276000
6,-0.084794,-1.016629,0.205400


In [33]:
#对reindexing有效的那些插值方法也可用于fillna
df = pd.DataFrame(np.random.randn(6,3))
df.iloc[2:, 1] = NA
df.iloc[4:, 2] = NA
print(df)
print(df.fillna(method='ffill'))
print(df.fillna(method='ffill', limit=2))

          0         1         2
0  0.072448  1.609753 -0.439802
1 -0.407405 -0.652035 -0.385677
2 -0.677603       NaN -0.073656
3  0.183035       NaN  0.215251
4  0.142637       NaN       NaN
5 -0.042959       NaN       NaN
          0         1         2
0  0.072448  1.609753 -0.439802
1 -0.407405 -0.652035 -0.385677
2 -0.677603 -0.652035 -0.073656
3  0.183035 -0.652035  0.215251
4  0.142637 -0.652035  0.215251
5 -0.042959 -0.652035  0.215251
          0         1         2
0  0.072448  1.609753 -0.439802
1 -0.407405 -0.652035 -0.385677
2 -0.677603 -0.652035 -0.073656
3  0.183035 -0.652035  0.215251
4  0.142637       NaN  0.215251
5 -0.042959       NaN  0.215251


In [34]:
data = pd.Series([1., NA, 3.5, NA, 7])
data.fillna(data.mean())

0    1.000000
1    3.833333
2    3.500000
3    3.833333
4    7.000000
dtype: float64

### 7.2 数据转换

#### 移除重复数据

In [35]:
data = pd.DataFrame({'k1':['one','two'] * 3 + ['two'],
                     'k2':[1, 1, 2, 3, 3, 4, 4]})
data

,k1,k2
0,one,1
1,two,1
2,one,2
3,two,3
4,one,3
5,two,4
6,two,4


In [36]:
data.duplicated() #返回一个布尔型Series，表示各行是否重复行
#(对前面出现的行)

0    False
1    False
2    False
3    False
4    False
5    False
6     True
dtype: bool

In [37]:
data.drop_duplicates()#返回一个DataFrame，重复的数组会标为False

,k1,k2
0,one,1
1,two,1
2,one,2
3,two,3
4,one,3
5,two,4


In [38]:
#默认都是根据全部列来进行重复项判断
#指定部分列进行重复项判断,根据K1列过滤重复项
data['v1'] = range(7)
print(data)
data.drop_duplicates(['k1'])

    k1  k2  v1
0  one   1   0
1  two   1   1
2  one   2   2
3  two   3   3
4  one   3   4
5  two   4   5
6  two   4   6


,k1,k2,v1
0,one,1,0
1,two,1,1


In [39]:
#duplicated和drop_duplicates默认保留的是第一个出现的的值的组合
#传入keep=’last‘则保留最后一个
data.drop_duplicates(['k1', 'k2'], keep='last')

,k1,k2,v1
0,one,1,0
1,two,1,1
2,one,2,2
3,two,3,3
4,one,3,4
6,two,4,6


#### 利用函数或映射进行数据转换

In [40]:
data = pd.DataFrame({'food': ['bacon', 'pulled pork', 'bacon',
                              'Pastrami', 'corned beef', 'Bacon',
                              'pastrami', 'honey ham', 'nova lox'],
                     'ounces': [4, 3, 12, 6, 7.5, 8, 3, 5, 6]})
data

,food,ounces
0,bacon,4.0
1,pulled pork,3.0
2,bacon,12.0
3,Pastrami,6.0
4,corned beef,7.5
5,Bacon,8.0
6,pastrami,3.0
7,honey ham,5.0
8,nova lox,6.0


In [41]:
meat_to_animal = {
  'bacon': 'pig',
  'pulled pork': 'pig',
  'pastrami': 'cow',
  'corned beef': 'cow',
  'honey ham': 'pig',
  'nova lox': 'salmon'
}

In [42]:
#Series的str.lower方法，将各个值转换为小写
lowercased = data['food'].str.lower()
lowercased

0          bacon
1    pulled pork
2          bacon
3       pastrami
4    corned beef
5          bacon
6       pastrami
7      honey ham
8       nova lox
Name: food, dtype: object

In [43]:
#Series的map方法可以接受一个函数或含有映射关系的字典型对象
data['animal'] = lowercased.map(meat_to_animal)
data

,food,ounces,animal
0,bacon,4.0,pig
1,pulled pork,3.0,pig
2,bacon,12.0,pig
3,Pastrami,6.0,cow
4,corned beef,7.5,cow
5,Bacon,8.0,pig
6,pastrami,3.0,cow
7,honey ham,5.0,pig
8,nova lox,6.0,salmon


In [44]:
data['food'].map(lambda x: meat_to_animal[x.lower()])
#形式有点难记！！！

0       pig
1       pig
2       pig
3       cow
4       cow
5       pig
6       cow
7       pig
8    salmon
Name: food, dtype: object

#### 替换值

In [45]:
data = pd.Series([1., -999., 2., -999., -1000., 3.])
data

0       1.0
1    -999.0
2       2.0
3    -999.0
4   -1000.0
5       3.0
dtype: float64

In [46]:
#-999这个值可能是一个表示缺失数据的标记值
#要将其替换为pandas能够理解的NA值
#我们可以利用replace来产生一个新的Series（除非传入inplace=True）
data.replace(-999, np.nan)

0       1.0
1       NaN
2       2.0
3       NaN
4   -1000.0
5       3.0
dtype: float64

In [47]:
#如果你希望一次性替换多个值
#可以传入一个由待替换值组成的列表以及一个替换值
data.replace([-999, -1000], np.nan)

0    1.0
1    NaN
2    2.0
3    NaN
4    NaN
5    3.0
dtype: float64

In [48]:
#要让每个值有不同的替换值，可以传递一个替换列表
data.replace([-999, -1000],[np.nan, 0])

0    1.0
1    NaN
2    2.0
3    NaN
4    0.0
5    3.0
dtype: float64

In [49]:
#传入的参数也可以是字典
data.replace({-999: np.nan, -1000: 0})
#data.replace方法与data.str.replace不同
#后者做的是字符串的元素级替换

0    1.0
1    NaN
2    2.0
3    NaN
4    0.0
5    3.0
dtype: float64

#### 重命名轴索引

In [67]:
data = pd.DataFrame(np.arange(12).reshape((3,4)),
                    index=['Ohio', 'Colorado', 'New York'],
                    columns=['one', 'two', 'three', 'four'])
data

,one,two,three,four
Ohio,0,1,2,3
Colorado,4,5,6,7
New York,8,9,10,11


In [68]:
#跟Series一样，轴索引也有一个map方法
transform = lambda x: x[:4].upper()
data.index.map(transform)

Index(['OHIO', 'COLO', 'NEW '], dtype='object')

In [69]:
data.index = data.index.map(transform)
data
#你可以将其赋值给index，这样就可以对DataFrame进行就地修改

,one,two,three,four
OHIO,0,1,2,3
COLO,4,5,6,7
NEW,8,9,10,11


In [70]:
#如果想要创建数据集的转换版（而不是修改原始数据）
#比较实用的方法是rename
data.rename(index=str.title, columns=str.upper)

,ONE,TWO,THREE,FOUR
Ohio,0,1,2,3
Colo,4,5,6,7
New,8,9,10,11


In [73]:
#rename可以结合字典型对象实现对部分轴标签的更新
#未修改原始数据
data.rename(index={'OHIO': 'INDIANA'},
            columns={'three': 'peekaboo'})

,one,two,three,four
OHIO,0,1,2,3
COLO,4,5,6,7
NEW,8,9,10,11


In [74]:
#rename可以实现复制DataFrame并对其索引和列标签进行赋值
#如果希望就地修改某个数据集，传入inplace=True即可
data.rename(index={'OHIO': 'INDIANA'}, inplace=True)
data

,one,two,three,four
INDIANA,0,1,2,3
COLO,4,5,6,7
NEW,8,9,10,11


#### 离散化和面元划分
为了便于分析，连续数据常常被离散化或拆分为“面元”（bin）

In [75]:
ages = [20, 22, 25, 27, 21, 23, 37, 31, 61, 45, 41, 32]

In [76]:
#将这些数据划分为“18到25”、“26到35”、“35到60”以及“60以上”几个面元
#使用pandas的cut函数
bins = [18, 25, 35, 60, 100]
cats = pd.cut(ages,bins)
cats
#pandas返回的是一个特殊的Categorical对象
#结果展示了pandas.cut划分的面元
#你可以将其看做一组表示面元名称的字符串
#它的底层含有一个表示不同分类名称的类型数组
#以及一个codes属性中的年龄数据的标签

[(18, 25], (18, 25], (18, 25], (25, 35], (18, 25], ..., (25, 35], (60, 100], (35, 60], (35, 60], (25, 35]]
Length: 12
Categories (4, interval[int64]): [(18, 25] < (25, 35] < (35, 60] < (60, 100]]

In [77]:
print(cats.codes)
print(cats.categories)
print(pd.value_counts(cats))

[0 0 0 1 0 0 2 1 3 2 2 1]
IntervalIndex([(18, 25], (25, 35], (35, 60], (60, 100]]
              closed='right',
              dtype='interval[int64]')
(18, 25]     5
(35, 60]     3
(25, 35]     3
(60, 100]    1
dtype: int64


In [78]:
#跟“区间”的数学符号一样，圆括号表示开端，而方括号则表示闭端（包括）
#哪边是闭端可以通过right=False进行修改
pd.cut(ages, [18, 26, 36, 61, 100], right=False)

[[18, 26), [18, 26), [18, 26), [26, 36), [18, 26), ..., [26, 36), [61, 100), [36, 61), [36, 61), [26, 36)]
Length: 12
Categories (4, interval[int64]): [[18, 26) < [26, 36) < [36, 61) < [61, 100)]

In [79]:
#你可以通过传递一个列表或数组到labels，设置自己的面元名称
group_names = ['Youth', 'YoungAdult', 'MiddleAged', 'Senior']
pd.cut(ages, bins, labels=group_names)

[Youth, Youth, Youth, YoungAdult, Youth, ..., YoungAdult, Senior, MiddleAged, MiddleAged, YoungAdult]
Length: 12
Categories (4, object): [Youth < YoungAdult < MiddleAged < Senior]

In [82]:
#如果向cut传入的是面元的数量而不是确切的面元边界
#则它会根据数据的最小值和最大值计算等长面元
#我们将一些均匀分布的数据分成四组
data = np.random.rand(20)
print(data)
pd.cut(data, 4, precision=2)
#选项precision=2，限定小数只有两位

[ 0.32371531  0.13981177  0.11584357  0.03854323  0.71083321  0.99233189
  0.66646135  0.93036257  0.46873233  0.39495714  0.50089409  0.09101766
  0.80302607  0.74306844  0.5877505   0.27535489  0.80455098  0.13211288
  0.6034298   0.96524935]


[(0.28, 0.52], (0.038, 0.28], (0.038, 0.28], (0.038, 0.28], (0.52, 0.75], ..., (0.038, 0.28], (0.75, 0.99], (0.038, 0.28], (0.52, 0.75], (0.75, 0.99]]
Length: 20
Categories (4, interval[float64]): [(0.038, 0.28] < (0.28, 0.52] < (0.52, 0.75] < (0.75, 0.99]]

In [83]:
#qcut是一个非常类似于cut的函数
#它可以根据样本分位数对数据进行面元划分
#根据数据的分布情况
#cut可能无法使各个面元中含有相同数量的数据点
#而qcut由于使用的是样本分位数，因此可以得到大小基本相等的面元
data = np.random.randn(1000)
cats = pd.qcut(data, 4)
print(cats)
print(pd.value_counts(cats))

[(-2.631, -0.719], (-0.0288, 0.727], (-0.0288, 0.727], (-2.631, -0.719], (-2.631, -0.719], ..., (-2.631, -0.719], (0.727, 3.408], (-2.631, -0.719], (-0.719, -0.0288], (-0.719, -0.0288]]
Length: 1000
Categories (4, interval[float64]): [(-2.631, -0.719] < (-0.719, -0.0288] < (-0.0288, 0.727] < (0.727, 3.408]]
(0.727, 3.408]       250
(-0.0288, 0.727]     250
(-0.719, -0.0288]    250
(-2.631, -0.719]     250
dtype: int64


In [88]:
#与cut类似，你也可以传递自定义的分位数（0到1之间的数值，包含端点）
cat = pd.qcut(data, [0, 0.1, 0.5, 0.9, 1.])
print(pd.value_counts(cat))
cat
#为什么不能根据传入的面元来划分？？？？？？？

(-0.0288, 1.294]     400
(-1.294, -0.0288]    400
(1.294, 3.408]       100
(-2.631, -1.294]     100
dtype: int64


[(-2.631, -1.294], (-0.0288, 1.294], (-0.0288, 1.294], (-1.294, -0.0288], (-1.294, -0.0288], ..., (-1.294, -0.0288], (-0.0288, 1.294], (-1.294, -0.0288], (-1.294, -0.0288], (-1.294, -0.0288]]
Length: 1000
Categories (4, interval[float64]): [(-2.631, -1.294] < (-1.294, -0.0288] < (-0.0288, 1.294] < (1.294, 3.408]]

#### 检测和过滤异常值

In [89]:
data = pd.DataFrame(np.random.randn(1000, 4))
data.describe()

,0,1,2,3
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,0.013509,-0.049550,0.031347,-0.001968
std,1.007299,1.009911,1.009413,0.984925
min,-3.576328,-2.825801,-2.816359,-3.198931
25%,-0.661388,-0.751834,-0.646775,-0.662610
50%,-0.028333,-0.052545,0.038200,-0.012499
75%,0.712433,0.667968,0.713136,0.668127
max,3.566891,2.982484,2.960869,3.017366


In [94]:
#要找出某列中绝对值大小超过3的值
col = data[2]
col[np.abs(col) > 2]

34     2.940911
72     2.265586
87    -2.416011
88    -2.106539
102   -2.558720
134   -2.196460
227    2.173227
247   -2.789208
270    2.087099
279    2.039880
307   -2.796330
336   -2.471477
346    2.159019
380    2.167346
404   -2.388742
455    2.115045
490   -2.444615
496    2.072590
500    2.007826
508    2.041274
572   -2.165185
589    2.177644
594    2.173889
608    2.317500
610   -2.440089
621    2.335108
623   -2.816359
650   -2.114318
666   -2.380126
679   -2.482957
685   -2.113557
695    2.665843
726    2.951000
747   -2.658738
767    2.117879
796    2.164375
800    2.174294
823    2.596175
829    2.144284
832    2.960869
864   -2.299955
879   -2.642855
893    2.351828
906    2.728690
916    2.536691
933   -2.445484
960   -2.023190
979   -2.184998
993   -2.412788
998   -2.186846
Name: 2, dtype: float64

In [95]:
#要选出全部含有“超过3或－3的值”的行
#你可以在布尔型DataFrame中使用any方法
data[(np.abs(data) > 3).any(1)]

,0,1,2,3
40,1.869174,-0.544584,0.500107,3.017366
260,3.566891,1.078813,0.277279,-0.825344
307,3.231843,-0.223847,-2.796330,1.248348
518,-3.058980,0.848385,0.980887,1.026293
567,3.260853,1.637692,-0.999823,0.790410
720,3.182216,0.279699,0.416698,0.398261
955,-3.576328,-0.114186,-1.830282,1.752394
966,-0.898952,0.727784,-0.736847,-3.198931


In [97]:
#根据这些条件，就可以对值进行设置
#下面的代码可以将值限制在区间－3到3以内
data[np.abs(data) > 3] = np.sign(data) * 3
data.describe()
#sign的作用就是取符号
#绝对值大于3的数就取其符号，-1或1然后乘上3

,0,1,2,3
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,0.012903,-0.049550,0.031347,-0.001787
std,1.001290,1.009911,1.009413,0.984246
min,-3.000000,-2.825801,-2.816359,-3.000000
25%,-0.661388,-0.751834,-0.646775,-0.662610
50%,-0.028333,-0.052545,0.038200,-0.012499
75%,0.712433,0.667968,0.713136,0.668127
max,3.000000,2.982484,2.960869,3.000000


In [98]:
np.sign(data).head()

,0,1,2,3
0,1.0,-1.0,-1.0,1.0
1,-1.0,-1.0,1.0,-1.0
2,-1.0,1.0,1.0,1.0
3,-1.0,-1.0,1.0,1.0
4,-1.0,1.0,-1.0,1.0


#### 排列和随机采样

In [100]:
df = pd.DataFrame(np.arange(5 * 4).reshape((5, 4)))
print(df)
sampler = np.random.permutation(5)
print(sampler)
#利用numpy.random.permutation函数可以轻松实现对Series
#或DataFrame的列的排列工作
#通过需要排列的轴的长度调用permutation
#可产生一个表示新顺序的整数数组

    0   1   2   3
0   0   1   2   3
1   4   5   6   7
2   8   9  10  11
3  12  13  14  15
4  16  17  18  19
[4 1 3 2 0]


In [101]:
print(df)
print(df.take(sampler))
#然后就可以在基于iloc的索引操作或take函数中使用该数组了

    0   1   2   3
0   0   1   2   3
1   4   5   6   7
2   8   9  10  11
3  12  13  14  15
4  16  17  18  19
    0   1   2   3
4  16  17  18  19
1   4   5   6   7
3  12  13  14  15
2   8   9  10  11
0   0   1   2   3


In [102]:
#如果不想用替换的方式选取随机子集，可以在Series
#和DataFrame上使用sample方法
df.sample(n=3)

,0,1,2,3
4,16,17,18,19
1,4,5,6,7
2,8,9,10,11


In [103]:
#要通过替换的方式产生样本（允许重复选择）
#可以传递replace=True到sample
choices = pd.Series([5, 7, -1, 6, 4])
print(choices)
draws = choices.sample(n=10, replace=True)
print(draws)

0    5
1    7
2   -1
3    6
4    4
dtype: int64
1    7
0    5
2   -1
0    5
3    6
4    4
2   -1
0    5
1    7
3    6
dtype: int64


#### 计算指标/哑变量

In [105]:
#另一种常用于统计建模或机器学习的转换方式是
#将分类变量（categorical variable）转换为“哑变量”或“指标矩阵”
#如果DataFrame的某一列中含有k个不同的值
#则可以派生出一个k列矩阵或DataFrame（其值全为1和0）
#pandas有一个get_dummies函数可以实现该功能
df = pd.DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'b'],
                    'data1': range(6)})
print(df)
pd.get_dummies(df['key'])

   data1 key
0      0   b
1      1   b
2      2   a
3      3   c
4      4   a
5      5   b


,a,b,c
0,0,1,0
1,0,1,0
2,1,0,0
3,0,0,1
4,1,0,0
5,0,1,0


In [106]:
#有时候，你可能想给指标DataFrame的列加上一个前缀
#以便能够跟其他数据进行合并
#get_dummies的prefix参数可以实现该功能
dummies = pd.get_dummies(df['key'], prefix='key')
df_with_dummy = df[['data1']].join(dummies)
df_with_dummy

,data1,key_a,key_b,key_c
0,0,0,1,0
1,1,0,1,0
2,2,1,0,0
3,3,0,0,1
4,4,1,0,0
5,5,0,1,0


In [107]:
#如果DataFrame中的某行同属于多个分类，则事情就会有点复杂。

mnames = ['movie_id', 'title', 'genres']
movies = pd.read_table('datasets/movielens/movies.dat', sep='::',
                        header=None, names=mnames)
movies[:10]

/Users/chenjin/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """


,movie_id,title,genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy
5,6,Heat (1995),Action|Crime|Thriller
6,7,Sabrina (1995),Comedy|Romance
7,8,Tom and Huck (1995),Adventure|Children's
8,9,Sudden Death (1995),Action
9,10,GoldenEye (1995),Action|Adventure|Thriller


In [110]:
#要为每个genre添加指标变量就需要做一些数据规整操作
#首先，我们从数据集中抽取出不同的genre值
all_genres = []
for x in movies.genres:
    all_genres.extend(x.split('|'))

genres = pd.unique(all_genres)

In [111]:
genres

array(['Animation', "Children's", 'Comedy', 'Adventure', 'Fantasy',
       'Romance', 'Drama', 'Action', 'Crime', 'Thriller', 'Horror',
       'Sci-Fi', 'Documentary', 'War', 'Musical', 'Mystery', 'Film-Noir',
       'Western'], dtype=object)

In [115]:
#构建指标DataFrame的方法之一是从一个全零DataFrame开始
zero_matrix = np.zeros((len(movies), len(genres)))
dummies = pd.DataFrame(zero_matrix, columns=genres)

In [116]:
#现在，迭代每一部电影，并将dummies各行的条目设为1
#要这么做，我们使用dummies.columns来计算每个类型的列索引
gen = movies.genres[0]
print(gen.split('|'))

dummies.columns.get_indexer(gen.split('|'))

['Animation', "Children's", 'Comedy']


array([0, 1, 2])

In [122]:
for i, gen in enumerate(movies.genres):
    indices = dummies.columns.get_indexer(gen.split('|'))
    dummies.iloc[i, indices] = 1
# print(dummies)

In [120]:
movies_windic = movies.join(dummies.add_prefix('Genre_'))
movies_windic.iloc[0]
# print(movies_windic)

movie_id                                       1
title                           Toy Story (1995)
genres               Animation|Children's|Comedy
Genre_Animation                                1
Genre_Children's                               1
Genre_Comedy                                   1
Genre_Adventure                                0
Genre_Fantasy                                  0
Genre_Romance                                  0
Genre_Drama                                    0
Genre_Action                                   0
Genre_Crime                                    0
Genre_Thriller                                 0
Genre_Horror                                   0
Genre_Sci-Fi                                   0
Genre_Documentary                              0
Genre_War                                      0
Genre_Musical                                  0
Genre_Mystery                                  0
Genre_Film-Noir                                0
Genre_Western       

In [123]:
#一个对统计应用有用的秘诀是
#结合get_dummies和诸如cut之类的离散化函数
np.random.seed(12345)
values = np.random.randn(10)
print(values)

bins = [0, 0.2, 0.4, 0.6, 0.8, 1]

pd.get_dummies(pd.cut(values, bins))

[-0.20470766  0.47894334 -0.51943872 -0.5557303   1.96578057  1.39340583
  0.09290788  0.28174615  0.76902257  1.24643474]


,"(0.0, 0.2]","(0.2, 0.4]","(0.4, 0.6]","(0.6, 0.8]","(0.8, 1.0]"
0,0,0,0,0,0
1,0,0,1,0,0
2,0,0,0,0,0
3,0,0,0,0,0
4,0,0,0,0,0
5,0,0,0,0,0
6,1,0,0,0,0
7,0,1,0,0,0
8,0,0,0,1,0
9,0,0,0,0,0


### 7.3 字符串操作

#### 字符串对象方法

In [124]:
val = 'a,b, guido'
val.split(',')

['a', 'b', ' guido']

In [125]:
#split常常与strip一起使用，以去除空白符（包括换行符）
pieces = [x.strip() for x in val.split(',')]
pieces

['a', 'b', 'guido']

In [126]:
#利用加法，可以将这些子字符串以双冒号分隔符的形式连接起来
first, second, third = pieces

first + '::' + second + '::' + third

'a::b::guido'

In [127]:
#但这种方式并不是很实用
#一种更快更符合Python风格的方式是
#向字符串"::"的join方法传入一个列表或元组
'::'.join(pieces)

'a::b::guido'

In [128]:
#其它方法关注的是子串定位
#检测子串的最佳方式是利用Python的in关键字，还可以使用index和find
print('guido' in val)
print(val.index(','))
print(val.find(':'))

True
1
-1


In [129]:
#注意find和index的区别：如果找不到字符串，index将会引发一个异常
val.index(':')

ValueError: substring not found

In [130]:
#与此相关，count可以返回指定子串的出现次数
val.count(',')

2

In [131]:
print(val.replace(',', '::'))
print(val.replace(',',''))

a::b:: guido
ab guido


#### 正则表达式

In [3]:
import re
#描述一个或多个空白符的regex是\s+
#假设我想要拆分一个字符串
#分隔符为数量不定的一组空白符（制表符、空格、换行符等）

text = "foo    bar\t baz  \tqux"
re.split('\s+', text)

['foo', 'bar', 'baz', 'qux']

In [133]:
#调用re.split('\s+',text)时，正则表达式会先被编译，
#然后再在text上调用其split方法。
#你可以用re.compile自己编译regex以得到一个可重用的regex对象

regex = re.compile('\s+')

regex.split(text)

['foo', 'bar', 'baz', 'qux']

In [134]:
#如果只希望得到匹配regex的所有模式，则可以使用findall方法
regex.findall(text)
#如果打算对许多字符串应用同一条正则表达式，
#强烈建议通过re.compile创建regex对象。这样将可以节省大量的CPU时间

['    ', '\t ', '  \t']

In [4]:
#findall返回的是字符串中所有的匹配项
#而search则只返回第一个匹配项
#match更加严格，它只匹配字符串的首部
text = """Dave dave@google.com
Steve steve@gmail.com
Rob rob@gmail.com
Ryan ryan@yahoo.com
"""
pattern = r'[A-Z0-9._%+-]+@[A-Z0-9.-]+\.[A-Z]{2,4}'

# re.IGNORECASE makes the regex case-insensitive
regex = re.compile(pattern, flags=re.IGNORECASE)

In [5]:
#对text使用findall将得到一组电子邮件地址
regex.findall(text)

['dave@google.com', 'steve@gmail.com', 'rob@gmail.com', 'ryan@yahoo.com']

In [6]:
#search返回的是文本中第一个电子邮件地址,以特殊的匹配项对象形式返回
m = regex.search(text)

print(m)
#对于上面那个regex，匹配项对象只能告诉我们模式
#在原字符串中的起始和结束位置
text[m.start():m.end()]

<_sre.SRE_Match object; span=(5, 20), match='dave@google.com'>


'dave@google.com'

In [7]:
#regex.match则将返回None，因为它只匹配出现在字符串开头的模式
print(regex.match(text))

None


In [8]:
#sub方法可以将匹配到的模式替换为指定字符串，并返回所得到的新字符串
print(regex.sub('REDACTED', text))

Dave REDACTED
Steve REDACTED
Rob REDACTED
Ryan REDACTED



In [9]:
#假设你不仅想要找出电子邮件地址，还想将各个地址分成3个部分
#用户名、域名以及域后缀。要实现此功能
#只需将待分段的模式的各部分用圆括号包起来即可
pattern = r'([A-Z0-9._%+-]+)@([A-Z0-9.-]+)\.([A-Z]{2,4})'

regex = re.compile(pattern, flags=re.IGNORECASE)

In [10]:
#由这种修改过的正则表达式所产生的匹配项对象
#可以通过其groups方法返回一个由模式各段组成的元组
m = regex.match('wesm@bright.net')

m.groups()

('wesm', 'bright', 'net')

In [11]:
#对于带有分组功能的模式，findall会返回一个元组列表
regex.findall(text)

[('dave', 'google', 'com'),
 ('steve', 'gmail', 'com'),
 ('rob', 'gmail', 'com'),
 ('ryan', 'yahoo', 'com')]

In [12]:
#sub还能通过诸如\1、\2之类的特殊符号访问各匹配项中的分组
#符号\1对应第一个匹配的组，\2对应第二个匹配的组，以此类推
print(regex.sub(r'Username: \1, Domain: \2, Suffix: \3', text))

Dave Username: dave, Domain: google, Suffix: com
Steve Username: steve, Domain: gmail, Suffix: com
Rob Username: rob, Domain: gmail, Suffix: com
Ryan Username: ryan, Domain: yahoo, Suffix: com



#### pandas的矢量化字符串函数
清理待分析的散乱数据时，常常需要做一些字符串规整化工作<br />
更为复杂的情况是，含有字符串的列有时还含有缺失数据

In [13]:
data = {'Dave': 'dave@google.com', 'Steve': 'steve@gmail.com',
        'Rob': 'rob@gmail.com', 'Wes': np.nan}
data = pd.Series(data)
print(data)
print(data.isnull())

Dave     dave@google.com
Rob        rob@gmail.com
Steve    steve@gmail.com
Wes                  NaN
dtype: object
Dave     False
Rob      False
Steve    False
Wes       True
dtype: bool


In [14]:
data.str.contains('gmail')

Dave     False
Rob       True
Steve     True
Wes        NaN
dtype: object

In [15]:
#也可以使用正则表达式，还可以加上任意re选项
print(pattern)
data.str.findall(pattern, flags=re.IGNORECASE)

([A-Z0-9._%+-]+)@([A-Z0-9.-]+)\.([A-Z]{2,4})


Dave     [(dave, google, com)]
Rob        [(rob, gmail, com)]
Steve    [(steve, gmail, com)]
Wes                        NaN
dtype: object

In [16]:
matches = data.str.match(pattern, flags=re.IGNORECASE)
matches

Dave     True
Rob      True
Steve    True
Wes       NaN
dtype: object

In [17]:
print(matches.str.get(1))
print(matches.str[0])

Dave    NaN
Rob     NaN
Steve   NaN
Wes     NaN
dtype: float64
Dave    NaN
Rob     NaN
Steve   NaN
Wes     NaN
dtype: float64


In [18]:
data.str[:5]

Dave     dave@
Rob      rob@g
Steve    steve
Wes        NaN
dtype: object

## 8 数据规整：聚合、合并和重塑
### 8.1 层次化索引

In [20]:
data = pd.Series(np.random.randn(),
                 index=[['a','a','a','b','b','c','c','d','d'],
                        [1, 2, 3, 1, 3, 1, 2, 2, 3]])
data

a  1   -0.189063
   2   -0.189063
   3   -0.189063
b  1   -0.189063
   3   -0.189063
c  1   -0.189063
   2   -0.189063
d  2   -0.189063
   3   -0.189063
dtype: float64

In [21]:
data.index

MultiIndex(levels=[['a', 'b', 'c', 'd'], [1, 2, 3]],
           labels=[[0, 0, 0, 1, 1, 2, 2, 3, 3], [0, 1, 2, 0, 2, 0, 1, 1, 2]])

In [22]:
print(data['b'])
print(data['b':'c'])
print(data.loc[['b','c']])

1   -0.189063
3   -0.189063
dtype: float64
b  1   -0.189063
   3   -0.189063
c  1   -0.189063
   2   -0.189063
dtype: float64
b  1   -0.189063
   3   -0.189063
c  1   -0.189063
   2   -0.189063
dtype: float64


In [23]:
#有时甚至还可以在“内层”中进行选取
data.loc[:, 2]

a   -0.189063
c   -0.189063
d   -0.189063
dtype: float64

In [24]:
#层次化索引在数据重塑和基于分组的操作
#（如透视表生成）中扮演着重要的角色
#例如，可以通过unstack方法将这段数据重新安排到一个DataFrame中
data.unstack()

,1,2,3
a,-0.189063,-0.189063,-0.189063
b,-0.189063,NaN,-0.189063
c,-0.189063,-0.189063,NaN
d,NaN,-0.189063,-0.189063


In [25]:
data.unstack().stack()

a  1   -0.189063
   2   -0.189063
   3   -0.189063
b  1   -0.189063
   3   -0.189063
c  1   -0.189063
   2   -0.189063
d  2   -0.189063
   3   -0.189063
dtype: float64

In [27]:
#对于一个DataFrame，每条轴都可以有分层索引
frame = pd.DataFrame(np.arange(12).reshape((4, 3)),
                     index=[['a','a','b','b'],[1, 2, 1, 2]],
                     columns=[['Ohio', 'Ohio', 'Colorado'],
                              ['Green', 'Red', 'Green']])
frame

Ohio     Colorado
    Green Red    Green
a 1     0   1        2
  2     3   4        5
b 1     6   7        8
  2     9  10       11

In [28]:
frame.index.names = ['key1', 'key2']

frame.columns.names = ['state', 'color']
frame

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

In [29]:
frame['Ohio']

color      Green  Red
key1 key2            
a    1         0    1
     2         3    4
b    1         6    7
     2         9   10

In [30]:
#可以单独创建MultiIndex然后复用
#上面那个DataFrame中的（带有分级名称）列可以这样创建
MultiIndex.from_arrays([['Ohio', 'Ohio', 'Colorado'], ['Green', 'Red', 'Green']],
                       names=['state', 'color'])

NameError: name 'MultiIndex' is not defined

#### 重排与分级排序

In [31]:
frame.swaplevel('key1', 'key2')

state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
2    a        3   4        5
1    b        6   7        8
2    b        9  10       11

In [35]:
print(frame.sort_index(level=1))
print(frame.swaplevel(0, 1).sort_index(level=0))

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
b    1        6   7        8
a    2        3   4        5
b    2        9  10       11
state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
     b        6   7        8
2    a        3   4        5
     b        9  10       11


#### 根据级别汇总统计

In [36]:
#许多对DataFrame和Series的描述和汇总统计都有一个level选项
#它用于指定在某条轴上求和的级别
#再以上面那个DataFrame为例，我们可以根据行或列上的级别来进行求和
print(frame.sum(level='key2'))

print(frame.sum(level='color', axis=1))
#这其实是利用了pandas的groupby功能

state  Ohio     Colorado
color Green Red    Green
key2                    
1         6   8       10
2        12  14       16
color      Green  Red
key1 key2            
a    1         2    1
     2         8    4
b    1        14    7
     2        20   10


#### 使用DataFrame的列进行索引

In [37]:
#人们经常想要将DataFrame的一个或多个列当做行索引来用
#或者可能希望将行索引变成DataFrame的列
frame = pd.DataFrame({'a': range(7), 'b':range(7, 0, -1),
                      'c':['one', 'one', 'one', 'two', 'two',
                           'two', 'two'],
                      'd':[0, 1, 2, 0, 1, 2, 3]})
frame

,a,b,c,d
0,0,7,one,0
1,1,6,one,1
2,2,5,one,2
3,3,4,two,0
4,4,3,two,1
5,5,2,two,2
6,6,1,two,3


In [38]:
#DataFrame的set_index函数会将其一个或多个列转换为行索引
#并创建一个新的DataFrame
frame2 = frame.set_index(['c', 'd'])
frame2

a  b
c   d      
one 0  0  7
    1  1  6
    2  2  5
two 0  3  4
    1  4  3
    2  5  2
    3  6  1

In [39]:
#默认情况下，那些列会从DataFrame中移除，但也可以将其保留下来
frame.set_index(['c', 'd'], drop=False)

a  b    c  d
c   d              
one 0  0  7  one  0
    1  1  6  one  1
    2  2  5  one  2
two 0  3  4  two  0
    1  4  3  two  1
    2  5  2  two  2
    3  6  1  two  3

In [40]:
#reset_index的功能跟set_index刚好相反
#层次化索引的级别会被转移到列里面
frame2.reset_index()

,c,d,a,b
0,one,0,0,7
1,one,1,1,6
2,one,2,2,5
3,two,0,3,4
4,two,1,4,3
5,two,2,5,2
6,two,3,6,1


### 8.2 合并数据集

#### 数据库风格的DataFrame合并
pandas.merge可根据一个或多个键将不同DataFrame中的行连接起来。<br />SQL或其他关系型数据库的用户对此应该会比较熟悉<br />因为它实现的就是数据库的join操作。<br />pandas.concat可以沿着一条轴将多个对象堆叠到一起<br />实例方法combine_first可以将重复数据拼接在一起<br />用一个对象中的值填充另一个对象中的缺失值。

In [42]:
df1 = pd.DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'a', 'b'],
                    'data1': range(7)})
df2 = pd.DataFrame({'key':['a', 'b', 'd'],
                    'data2': range(3)})
print(df1)
print(df2)

   data1 key
0      0   b
1      1   b
2      2   a
3      3   c
4      4   a
5      5   a
6      6   b
   data2 key
0      0   a
1      1   b
2      2   d


In [43]:
pd.merge(df1, df2)

,data1,key,data2
0,0,b,1
1,1,b,1
2,6,b,1
3,2,a,0
4,4,a,0
5,5,a,0


In [44]:
#如果没有指定，merge就会将重叠列的列名当做键
pd.merge(df1, df2, on='key')

,data1,key,data2
0,0,b,1
1,1,b,1
2,6,b,1
3,2,a,0
4,4,a,0
5,5,a,0


In [47]:
#如果两个对象的列名不同，也可以分别进行指定
df3 = pd.DataFrame({'lkey': ['b', 'b', 'a', 'c', 'a', 'a', 'b'],
                    'data1': range(7)})

df4 = pd.DataFrame({'rkey': ['a', 'b', 'd'],
                    'data2': range(3)})
print(df3)
print(df4)
pd.merge(df3 ,df4, left_on='lkey', right_on='rkey')
#可能你已经注意到了，结果里面c和d以及与之相关的数据消失了。
#默认情况下，merge做的是“内连接”；结果中的键是交集。
#其他方式还有"left"、"right"以及"outer"。
#外连接求取的是键的并集，组合了左连接和右连接的效果

   data1 lkey
0      0    b
1      1    b
2      2    a
3      3    c
4      4    a
5      5    a
6      6    b
   data2 rkey
0      0    a
1      1    b
2      2    d


,data1,lkey,data2,rkey
0,0,b,1,b
1,1,b,1,b
2,6,b,1,b
3,2,a,0,a
4,4,a,0,a
5,5,a,0,a


In [48]:
pd.merge(df1, df2, how='outer')

,data1,key,data2
0,0.0,b,1.0
1,1.0,b,1.0
2,6.0,b,1.0
3,2.0,a,0.0
4,4.0,a,0.0
5,5.0,a,0.0
6,3.0,c,NaN
7,NaN,d,2.0


In [51]:
df1 = pd.DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'b'],
                    'data1': range(6)})
df2 = pd.DataFrame({'key': ['a', 'b', 'a', 'b', 'd'],
                    'data2':range(5)})
print(df1)
print(df2)
print(pd.merge(df1, df2, on='key', how='left'))

   data1 key
0      0   b
1      1   b
2      2   a
3      3   c
4      4   a
5      5   b
   data2 key
0      0   a
1      1   b
2      2   a
3      3   b
4      4   d
    data1 key  data2
0       0   b    1.0
1       0   b    3.0
2       1   b    1.0
3       1   b    3.0
4       2   a    0.0
5       2   a    2.0
6       3   c    NaN
7       4   a    0.0
8       4   a    2.0
9       5   b    1.0
10      5   b    3.0


In [52]:
pd.merge(df1, df2, how='inner')

,data1,key,data2
0,0,b,1
1,0,b,3
2,1,b,1
3,1,b,3
4,5,b,1
5,5,b,3
6,2,a,0
7,2,a,2
8,4,a,0
9,4,a,2


In [56]:
left = pd.DataFrame({'key1': ['foo', 'foo', 'bar'],
                     'key2': ['one', 'two', 'one'],
                     'lval': [1, 2, 3]})

right = pd.DataFrame({'key1': ['foo', 'foo', 'bar', 'bar'],
                      'key2': ['one', 'one', 'one', 'two'],
                      'rval': [4, 5, 6, 7]})
print(left)
print(right)
pd.merge(left, right, on=['key1', 'key2'], how='outer')
#结果中会出现哪些键组合取决于所选的合并方式
#注意：在进行列－列连接时，DataFrame对象中的索引会被丢弃

  key1 key2  lval
0  foo  one     1
1  foo  two     2
2  bar  one     3
  key1 key2  rval
0  foo  one     4
1  foo  one     5
2  bar  one     6
3  bar  two     7


,key1,key2,lval,rval
0,foo,one,1.0,4.0
1,foo,one,1.0,5.0
2,foo,two,2.0,NaN
3,bar,one,3.0,6.0
4,bar,two,NaN,7.0


In [57]:
print(pd.merge(left, right, on='key1'))
pd.merge(left, right, on='key1', suffixes=('_left', '_right'))
#suffixes的参数的作用

  key1 key2_x  lval key2_y  rval
0  foo    one     1    one     4
1  foo    one     1    one     5
2  foo    two     2    one     4
3  foo    two     2    one     5
4  bar    one     3    one     6
5  bar    one     3    two     7


,key1,key2_left,lval,key2_right,rval
0,foo,one,1,one,4
1,foo,one,1,one,5
2,foo,two,2,one,4
3,foo,two,2,one,5
4,bar,one,3,one,6
5,bar,one,3,two,7


#### 索引上的合并

In [54]:
#有时候，DataFrame中的连接键位于其索引中。
#在这种情况下，你可以传入left_index=True或right_index=True
#（或两个都传）
#以说明索引应该被用作连接键
left1 = pd.DataFrame({'key': ['a', 'b', 'a', 'a', 'b', 'c'],
                      'value': range(6)})
right1 = pd.DataFrame({'group_val': [3.5, 7]}, index=['a', 'b'])

print(left1)
print(right1)
print(pd.merge(left1, right1, left_on='key', right_index=True))

  key  value
0   a      0
1   b      1
2   a      2
3   a      3
4   b      4
5   c      5
   group_val
a        3.5
b        7.0
  key  value  group_val
0   a      0        3.5
2   a      2        3.5
3   a      3        3.5
1   b      1        7.0
4   b      4        7.0


In [58]:
#由于默认的merge方法是求取连接键的交集
#因此你可以通过外连接的方式得到它们的并集
pd.merge(left1, right1, left_on='key', right_index=True, how='outer')

,key,value,group_val
0,a,0,3.5
2,a,2,3.5
3,a,3,3.5
1,b,1,7.0
4,b,4,7.0
5,c,5,NaN


In [59]:
lefth = pd.DataFrame({'key1': ['Ohio', 'Ohio', 'Ohio',
                               'Nevada', 'Nevada'],
                      'key2': [2000, 2001, 2002, 2001, 2002],
                      'data': np.arange(5.)})
righth = pd.DataFrame(np.arange(12).reshape((6, 2)),
                      index=[['Nevada', 'Nevada', 'Ohio', 'Ohio',
                              'Ohio', 'Ohio'],
                             [2001, 2000, 2000, 2000, 2001, 2002]],
                     columns=['event1', 'event2'])
print(lefth)
print(righth)

   data    key1  key2
0   0.0    Ohio  2000
1   1.0    Ohio  2001
2   2.0    Ohio  2002
3   3.0  Nevada  2001
4   4.0  Nevada  2002
             event1  event2
Nevada 2001       0       1
       2000       2       3
Ohio   2000       4       5
       2000       6       7
       2001       8       9
       2002      10      11


In [ ]:
pd.merge(lefth, righth, left_on=['key1', 'key2'], right_index=True)

In [ ]:
pd.merge(lefth, righth, left_on=['key1', 'key2'],
         right_index=True, how='outer')

In [ ]:
left2 = pd.DataFrame([[1., 2.], [3., 4.], [5., 6.]],
                     index=['a', 'c', 'e'],
                     columns=['Ohio', 'Nevada'])
right2 = pd.DataFrame([[7., 8.], [9., 10.], [11., 12.], [13, 14]],
                      index=['b', 'c', 'd', 'e'],
                      columns=['Missouri', 'Alabama'])
print(left2)
print(right2)
print(pd.merge(left2,right2, how='outer', left_index=True, right_index=True))

In [ ]:
left2.join(right2, how='outer')

In [ ]:
left1.join(right1, on='key')

In [ ]:
another = pd.DataFrame([[7., 8.], [9., 10.], [11., 12], [16., 17.]],
                       index=['a', 'c', 'e', 'f'],
                       columns=['New York', 'Oregon'])
print(another)
print(left2.join([right2, another]))
printt(left2.join([right2, another], how='outer'))

#### 轴向连接

In [ ]:
arr = np.arange(12).reshape((3, 4))
print(arr)

print(np.concatenate([arr, arr], axis=1))

In [ ]:
s1 = pd.Series([0, 1], index=['a', 'b'])

s2 = pd.Series([2, 3, 4],index=['c', 'd', 'e'])

s3 = pd.Series([5, 6], index=['f', 'g'])

In [ ]:
pd.concat([s1, s2, s3])

In [ ]:
pd.concat([s1, s2, s3], axis=1)

In [ ]:
s4 = pd.concat([s1, s3])
print(s4)
print(pd.concat([s1, s4], axis=1))
print(pd.concat([s1, s4], axis=1, join='inner'))

In [ ]:
pd.concat([s1, s3], axis=1, join_axes=[['a', 'c', 'b', 'e']])

In [ ]:
result = pd.concat([s1, s1, s3], keys=['one','two','three'])
print(result)
print(result.unstack())

In [ ]:
pd.concat([s1, s2, s3], axis=1, key=['one','two','three'])

In [ ]:
df1 = pd.DataFrame(np.arange(6).reshape(3,2), index=['a', 'b', 'c'],
                   column=['one', 'four'])
print(df1)
print(df2)
pd.concat([df1, df2], axis=1, keys=['level1', 'level2'])

In [ ]:
pd.concat({'level1':df1, 'level2':df2},axis=1)

In [ ]:
pd.concat([df1, df2], axis=1, keys=['level1', 'level2'],
          names=['upper', 'lower'])

In [ ]:
df1 = pd.DataFrame(np.random.randn(3, 4), columns=['a', 'b', 'c', 'd'])
df2 = pd.DataFrame(np.random.randn(2, 3), columns=['b', 'd', 'a'])
print(df1)
print(df1)

In [ ]:
pd.concat([df1, df2], ignore_index=True)

#### 合并重叠数据

### 8.3 重塑和轴向旋转

#### 重塑层次化索引

#### 将“长格式”旋转为“宽格式”

#### 将“宽格式”旋转为“长格式”